In [ ]:
特征：
   'label','popularity','carCommentVolum','newsReplyVolum'平移了12个月
    train:
        1,  2,  3,  4,  5,  6,  7,  8,  9,  10,  11,  12
        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,  23,  24 
        
    test: 21,22,23,24

In [1]:
# import sys
import numpy as np
import pandas as pd
# import os 
# import gc
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# path  = './ccf_car/'
train_sales  = pd.read_csv('train_sales_data_train.csv')
train_search = pd.read_csv('train_search_data_train.csv')
train_user   = pd.read_csv('train_user_reply_data_train.csv')
evaluation_public = pd.read_csv('evaluation_public_9.csv')
# submit_example    = pd.read_csv('submit_example.csv')
data = pd.concat([train_sales, evaluation_public], ignore_index=True)
data = data.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user, 'left', on=['model', 'regYear', 'regMonth'])
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])
#索引-->value
#LabelEncoder
for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
    #key-->value
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']
initial_key_list = ['adcode','bodyType','forecastVolum','id','model','province','regMonth','regYear','salesVolume','popularity','carCommentVolum','newsReplyVolum'	,'label','mt']
data = data[initial_key_list].copy()


data['model_weight'] = data.groupby('model')['label'].transform('mean')
data['n_label'] = data['label'] / data['model_weight']

data.tail()

,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum,label,mt,model_weight,n_label
31675,350000,0,0.0,5276,59,福建,12,2017,NaN,NaN,NaN,NaN,NaN,24,171.325,NaN
31676,210000,0,0.0,5277,59,辽宁,12,2017,NaN,NaN,NaN,NaN,NaN,24,171.325,NaN
31677,500000,0,0.0,5278,59,重庆,12,2017,NaN,NaN,NaN,NaN,NaN,24,171.325,NaN
31678,610000,0,0.0,5279,59,陕西,12,2017,NaN,NaN,NaN,NaN,NaN,24,171.325,NaN
31679,230000,0,0.0,5280,59,黑龙江,12,2017,NaN,NaN,NaN,NaN,NaN,24,171.325,NaN


In [3]:
def windows_feature(data,judge):
    if (judge >1):
        print("windows_var_features start")
        del data['windows4_var'],data['windows4_std'],data['windows5_var'],data['windows5_std'],data['windows6_var'],data['windows6_std'],data['model_adcode']
#         del data['ratio'],data['differ'],data['ratio2'],data['differ2']
        df = data.copy()
        df_all = pd.DataFrame()
        df['model_adcode'] = df['adcode'] + df['model']
        model_adcode = list(df.model_adcode.unique())
        for i in tqdm(model_adcode):
            idx = (df['model_adcode']==i)
            df2 = df[idx] 
            df2['windows4_var'] = df2['n_label'].rolling(4).var()
            df2['windows5_var'] = df2['n_label'].rolling(5).var()
            df2['windows6_var'] = df2['n_label'].rolling(6).var()
            
            df2['windows4_std'] = df2['n_label'].rolling(4).std()
            df2['windows5_std'] = df2['n_label'].rolling(5).std()
            df2['windows6_std'] = df2['n_label'].rolling(6).std()
            
#             df2['ratio'] = df2.groupby('model_adcode')['label'].apply(lambda x: x / x.shift(1)) #.apply(lambda x: x - x.shift(1)).reset_index()
#             df2['differ'] = df2.groupby('model_adcode')['label'].apply(lambda x: x - x.shift(1))
            
#             df2['ratio2'] = df2.groupby('model_adcode')['ratio'].apply(lambda x: x / x.shift(1)) #.apply(lambda x: x - x.shift(1)).reset_index()
#             df2['differ2'] = df2.groupby('model_adcode')['differ'].apply(lambda x: x - x.shift(1))
            
            df_all = pd.concat([df_all, df2], ignore_index=True)
        data = data.merge(df_all, 'left', on=['adcode','bodyType','forecastVolum','id','model','province','regMonth','regYear','salesVolume','popularity','carCommentVolum','newsReplyVolum','pred_label','label','mt','n_label','model_weight'])
        print("windows_var_features end")
        return data
    else:
        print("windows_var_features start")
        df = data.copy()
        df_all = pd.DataFrame()
        df['model_adcode'] = df['adcode'] + df['model']
        model_adcode = list(df.model_adcode.unique())
        for i in tqdm(model_adcode):
            idx = (df['model_adcode']==i)
            df2 = df[idx]

            df2['windows4_var'] = df2['n_label'].rolling(4).var()
            df2['windows5_var'] = df2['n_label'].rolling(5).var()
            df2['windows6_var'] = df2['n_label'].rolling(6).var()
            
            df2['windows4_std'] = df2['n_label'].rolling(4).std()
            df2['windows5_std'] = df2['n_label'].rolling(5).std()
            df2['windows6_std'] = df2['n_label'].rolling(6).std()
            
#             df2['ratio'] = df2.groupby('model_adcode')['label'].apply(lambda x: x / x.shift(1)) #.apply(lambda x: x - x.shift(1)).reset_index()
#             df2['differ'] = df2.groupby('model_adcode')['label'].apply(lambda x: x - x.shift(1))
#             df2['ratio2'] = df2.groupby('model_adcode')['ratio'].apply(lambda x: x / x.shift(1)) #.apply(lambda x: x - x.shift(1)).reset_index()
#             df2['differ2'] = df2.groupby('model_adcode')['differ'].apply(lambda x: x - x.shift(1))

            df_all = pd.concat([df_all, df2], ignore_index=True)
        data = data.merge(df_all, 'left', on=['adcode','bodyType','forecastVolum','id','model','province','regMonth','regYear','salesVolume','popularity','carCommentVolum','newsReplyVolum','label','mt','n_label','model_weight'])
        print("windows_var_features end")
        return data
        


In [6]:
def get_stat_feature(df_):   
    df = df_.copy()
    stat_feat = []
    df['model_adcode'] = df['adcode'] + df['model']
    df['model_adcode_mt'] = df['model_adcode'] * 100 + df['mt']
    for col in tqdm(['n_label','popularity']):#,'newsReplyVolum'
        # shift
        for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
            stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col,i))
            df['model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'] + i
            df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,i))
            df['shift_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])    
    return df,stat_feat

In [7]:
def valid_score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

In [8]:
def online_score(data, pred='pred_label', label='true_sales', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    y = pd.read_csv("valid_data_y.csv")
    data = data.merge(y, 'left', on=['id'])
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

In [9]:
def get_model_type(train_x,train_y,valid_x,valid_y,m_type='lgb'):   
    if m_type == 'lgb':
        model = lgb.LGBMRegressor(
                                num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
                                max_depth=-1, learning_rate=0.1, min_child_samples=5, random_state=500,
                                n_estimators=2000, subsample=0.9, colsample_bytree=0.7,
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              categorical_feature=cate_feat, 
              early_stopping_rounds=100, verbose=100)      
    elif m_type == 'xgb':
        model = xgb.XGBRegressor(
                                max_depth=5 , learning_rate=0.01, n_estimators=4000, 
                                objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                                colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)   
    return model

In [10]:
def get_train_model(df_, m, m_type):
    df = df_.copy()
    # 数据集划分
    st = 9
    all_idx   = (df['mt'].between(st , m-1))
    train_idx = (df['mt'].between(st , m-5))
    valid_idx = (df['mt'].between(m-4, m-4))
    test_idx  = (df['mt'].between(m  , m  ))
    
    print('all_idx  :',st ,m-1)
    print('train_idx:',st ,m-5)
    print('valid_idx:',m-4,m-4)
    print('test_idx :',m  ,m  )  
    # 最终确认
    train_x = df[train_idx][features]
    train_y = df[train_idx]['n_label']
    valid_x = df[valid_idx][features]
    valid_y = df[valid_idx]['n_label']   
    # get model
    model = get_model_type(train_x,train_y,valid_x,valid_y,m_type)  
    # offline
    df['pred_label'] = model.predict(df[features]) *df['model_weight']
    best_score = valid_score(df[valid_idx]) 
    # online
    if m_type == 'lgb':
        model.n_estimators = model.best_iteration_ + 100
        model.fit(df[all_idx][features], df[all_idx]['n_label'], categorical_feature=cate_feat)
    elif m_type == 'xgb':
        model.n_estimators = model.best_iteration + 100
        model.fit(df[all_idx][features], df[all_idx]['label'])
    df['forecastVolum'] = model.predict(df[features]) * df['model_weight']
    print('valid mean:',df[valid_idx]['pred_label'].mean())
    print('true  mean:',df[valid_idx]['label'].mean())
    print('test  mean:',df[test_idx]['forecastVolum'].mean())
    # 阶段结果
    sub = df[test_idx][['id']]
    sub['forecastVolum'] = df[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    print(sub.shape)
    return sub,df[valid_idx]['pred_label']

In [11]:
for month in [21,22,23,24]: 
    judge = month-20
    m_type = 'lgb' 
    data = windows_feature(data,judge)
    data_df, stat_feat = get_stat_feature(data)
    
    win_feat = ['windows4_var','windows4_std','windows5_var','windows5_std','windows6_var','windows6_std']
#     qushi_feat = ['ratio2','differ2']
    num_feat = ['regYear','regMonth'] + stat_feat +win_feat#,'windows_var','windows_std'
    cate_feat = ['adcode','bodyType','model']

    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))

    features = num_feat + cate_feat
    print(len(features), len(set(features)))
    sub,val_pred = get_train_model(data_df, month, m_type)
    online_test_idx  = (data_df['mt'].between(21  , 24  ))
    print("---")
    print(data.loc[(data.regMonth==month)].shape)
    data.loc[(data.mt==month), 'salesVolume'   ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'label'      ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values
    data['n_label'] = data['label'] / data['model_weight']


    # # sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
# # sub.columns = ['id','forecastVolum']
# # sub[['id','forecastVolum']].round().astype(int).to_csv('submit/yulao_lgb.csv', index=False)
best_score = online_score(data[online_test_idx]) 

windows_var_features start


100%|██████████████████████████████████████████████████████████████████████████████| 1320/1320 [00:27<00:00, 48.26it/s]


windows_var_features end


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.34it/s]


35 35
all_idx  : 9 20
train_idx: 9 16
valid_idx: 17 17
test_idx : 21 21
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 0.013988	valid_1's l2: 0.0307807
[200]	training's l2: 0.00731169	valid_1's l2: 0.0301776
[300]	training's l2: 0.00453902	valid_1's l2: 0.0296335
[400]	training's l2: 0.00309156	valid_1's l2: 0.0293086
[500]	training's l2: 0.00222681	valid_1's l2: 0.0293166
Early stopping, best iteration is:
[445]	training's l2: 0.00266495	valid_1's l2: 0.0292488
0.8177625487262247
valid mean: 510.5111241661089
true  mean: 534.5318181818182
test  mean: 576.507705636156
(1320, 2)
---
(0, 23)
windows_var_features start


...

35 35
all_idx  : 9 22
train_idx: 9 18
valid_idx: 19 19
test_idx : 23 23
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 0.0151371	valid_1's l2: 0.0840255
[200]	training's l2: 0.00835621	valid_1's l2: 0.0822735
[300]	training's l2: 0.00532646	valid_1's l2: 0.0812761
[400]	training's l2: 0.00370709	valid_1's l2: 0.0810626
[500]	training's l2: 0.00276381	valid_1's l2: 0.0807436
[600]	training's l2: 0.00211646	valid_1's l2: 0.0804997
[700]	training's l2: 0.00166733	valid_1's l2: 0.0803296
[800]	training's l2: 0.00132825	valid_1's l2: 0.08019
[900]	training's l2: 0.00107761	valid_1's l2: 0.0801533
Early stopping, best iteration is:
[866]	training's l2: 0.00114927	valid_1's l2: 0.0801043
0.7715109953501244
valid mean: 530.5037867361359
true  mean: 552.2295454545455
test  mean: 581.6263011433413
(1320, 2)
---
(0, 24)
windows_var_features start


100%|██████████████████████████████████████████████████████████████████████████████| 1320/1320 [00:25<00:00, 50.82it/s]


windows_var_features end


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.17it/s]


35 35
all_idx  : 9 23
train_idx: 9 19
valid_idx: 20 20
test_idx : 24 24
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 0.0160316	valid_1's l2: 0.0754248
Early stopping, best iteration is:
[46]	training's l2: 0.030339	valid_1's l2: 0.0714053
0.7827107203046785
valid mean: 571.1295754178078
true  mean: 620.0212121212121
test  mean: 625.5159964446401
(1320, 2)
---
(0, 24)
0.5980155112751024


In [186]:
for month in [21,22,23,24]: 
    m_type = 'xgb' 
    judge = month-20
    data = windows_feature(data,judge)
    
    data_df, stat_feat = get_stat_feature(data)
    
    win_feat = ['windows4_var','windows4_std','windows5_var','windows5_std','windows6_var','windows6_std']
#     qushi_feat = ['ratio2','differ2']
    num_feat = ['regYear','regMonth'] + stat_feat +win_feat#,'windows_var','windows_std'
    cate_feat = ['adcode','bodyType','model']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    online_test_idx  = (data_df['mt'].between(21  , 24  ))
    print("---")
    print(data.loc[(data.regMonth==month)].shape)
    data.loc[(data.mt==month), 'salesVolume'   ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'label'      ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values


    # # sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
# # sub.columns = ['id','forecastVolum']
# # sub[['id','forecastVolum']].round().astype(int).to_csv('submit/yulao_lgb.csv', index=False)
best_score = online_score(data[online_test_idx]) 


windows_var_features start




  0%|                                                                                         | 0/1320 [00:00<?, ?it/s]

  0%|▎                                                                                | 6/1320 [00:00<00:22, 57.85it/s]

  1%|▋                                                                               | 12/1320 [00:00<00:22, 57.68it/s]

  1%|█                                                                               | 18/1320 [00:00<00:22, 57.40it/s]

  2%|█▍                                                                              | 24/1320 [00:00<00:22, 56.72it/s]

  2%|█▊                                                                              | 30/1320 [00:00<00:22, 56.25it/s]

  3%|██▏                                                                             | 36/1320 [00:00<00:23, 55.31it/s]

  3%|██▌                                                                             | 42/1320 [00:00<00:23, 54.67it/s]

  4%|██▊                      

 59%|██████████████████████████████████████████████▍                                | 776/1320 [00:15<00:11, 47.96it/s]

 59%|██████████████████████████████████████████████▋                                | 781/1320 [00:15<00:11, 47.08it/s]

 60%|███████████████████████████████████████████████                                | 786/1320 [00:15<00:11, 46.68it/s]

 60%|███████████████████████████████████████████████▎                               | 791/1320 [00:15<00:11, 46.61it/s]

 60%|███████████████████████████████████████████████▋                               | 796/1320 [00:15<00:11, 47.21it/s]

 61%|███████████████████████████████████████████████▉                               | 801/1320 [00:15<00:10, 47.36it/s]

 61%|████████████████████████████████████████████████▏                              | 806/1320 [00:16<00:10, 47.62it/s]

 61%|████████████████████████████████████████████████▌                              | 811/1320 [00:16<00:10, 47.66it/s]

 62%|███████████████████████████

windows_var_features end




  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  2.47it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.01it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 28.65it/s]

...



  0%|                                                                                         | 0/1320 [00:00<?, ?it/s]

  0%|▎                                                                                | 6/1320 [00:00<00:23, 55.69it/s]

  1%|▋                                                                               | 12/1320 [00:00<00:23, 54.93it/s]

  1%|█                                                                               | 17/1320 [00:00<00:24, 53.23it/s]

  2%|█▍                                                                              | 23/1320 [00:00<00:24, 53.80it/s]

  2%|█▊                                                                              | 29/1320 [00:00<00:24, 53.10it/s]

  3%|██                                                                              | 35/1320 [00:00<00:24, 53.20it/s]

  3%|██▍                                                                             | 41/1320 [00:00<00:23, 54.59it/s]

  4%|██▊                      

 57%|█████████████████████████████████████████████▍                                 | 759/1320 [00:15<00:14, 39.95it/s]

 58%|█████████████████████████████████████████████▋                                 | 764/1320 [00:15<00:14, 38.97it/s]

 58%|█████████████████████████████████████████████▉                                 | 768/1320 [00:15<00:14, 38.40it/s]

 59%|██████████████████████████████████████████████▎                                | 773/1320 [00:15<00:13, 39.36it/s]

 59%|██████████████████████████████████████████████▌                                | 777/1320 [00:15<00:13, 38.89it/s]

 59%|██████████████████████████████████████████████▋                                | 781/1320 [00:16<00:14, 36.97it/s]

 59%|██████████████████████████████████████████████▉                                | 785/1320 [00:16<00:14, 37.22it/s]

 60%|███████████████████████████████████████████████▎                               | 790/1320 [00:16<00:13, 37.96it/s]

 60%|███████████████████████████

windows_var_features end




  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  2.26it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.00it/s]

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 27.85it/s]

35 35
all_idx  : 9 23
train_idx: 9 19
valid_idx: 20 20
test_idx : 24 24
[0]	validation_0-rmse:981.051	validation_1-rmse:1016.16
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:980.525	validation_1-rmse:1015.63
[200]	validation_0-rmse:979.105	validation_1-rmse:1014.21
[300]	validation_0-rmse:975.313	validation_1-rmse:1010.41
[400]	validation_0-rmse:965.461	validation_1-rmse:1000.55
[500]	validation_0-rmse:941.313	validation_1-rmse:976.4
[600]	validation_0-rmse:888.179	validation_1-rmse:923.159
[700]	validation_0-rmse:789.303	validation_1-rmse:823.709
[800]	validation_0-rmse:642.426	validation_1-rmse:676.553
[900]	validation_0-rmse:474.951	validation_1-rmse:511.82
[1000]	validation_0-rmse:327.363	validation_1-rmse:378.341
[1100]	validation_0-rmse:225.889	validation_1-rmse:296.488
[1200]	validation_0-rmse:172.961	validation_1-rmse:254.519
[1300]	validatio

In [14]:
for month in [21,22,23,24]: 
    m_type = 'xgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    win_feat = ['windows4_var','windows4_std','windows5_var','windows5_std','windows6_var','windows6_std']
#     qushi_feat = ['ratio2','differ2']
    num_feat = ['regYear','regMonth'] + stat_feat +win_feat#,'windows_var','windows_std'
    cate_feat = ['adcode','bodyType','model']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values	
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
sub[['id','forecastVolum']].round().astype(int).to_csv('submit/yulao_xgb.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 15.10it/s]


53 53
all_idx  : 13 24
train_idx: 13 20
valid_idx: 21 21
test_idx : 25 25
[0]	validation_0-rmse:841.387	validation_1-rmse:1046.81
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:840.854	validation_1-rmse:1046.28
[200]	validation_0-rmse:839.416	validation_1-rmse:1044.83
[300]	validation_0-rmse:835.586	validation_1-rmse:1040.97
[400]	validation_0-rmse:825.694	validation_1-rmse:1030.91
[500]	validation_0-rmse:801.738	validation_1-rmse:1006.18
[600]	validation_0-rmse:750.162	validation_1-rmse:951.587
[700]	validation_0-rmse:657.192	validation_1-rmse:849.31
[800]	validation_0-rmse:524.723	validation_1-rmse:696.311
[900]	validation_0-rmse:380.162	validation_1-rmse:522.151
[1000]	validation_0-rmse:259.234	validation_1-rmse:376.202
[1100]	validation_0-rmse:179.886	validation_1-rmse:282.214
[1200]	validation_0-rmse:145.639	validation_1-rmse:248.148
[1300]	valid

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 12.25it/s]


53 53
all_idx  : 13 25
train_idx: 13 21
valid_idx: 22 22
test_idx : 26 26
[0]	validation_0-rmse:866.621	validation_1-rmse:1007.38
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:866.089	validation_1-rmse:1006.85
[200]	validation_0-rmse:864.653	validation_1-rmse:1005.43
[300]	validation_0-rmse:860.828	validation_1-rmse:1001.62
[400]	validation_0-rmse:850.935	validation_1-rmse:991.753
[500]	validation_0-rmse:826.915	validation_1-rmse:967.795
[600]	validation_0-rmse:774.935	validation_1-rmse:915.746
[700]	validation_0-rmse:680.726	validation_1-rmse:819.708
[800]	validation_0-rmse:544.976	validation_1-rmse:682.224
[900]	validation_0-rmse:395.274	validation_1-rmse:534.143
[1000]	validation_0-rmse:268.46	validation_1-rmse:413.542
[1100]	validation_0-rmse:185.094	validation_1-rmse:338.885
[1200]	validation_0-rmse:148.412	validation_1-rmse:310.795
[1300]	valid

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 11.60it/s]


53 53
all_idx  : 13 26
train_idx: 13 22
valid_idx: 23 23
test_idx : 27 27
[0]	validation_0-rmse:881.709	validation_1-rmse:1071.9
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:881.177	validation_1-rmse:1071.36
[200]	validation_0-rmse:879.744	validation_1-rmse:1069.89
[300]	validation_0-rmse:875.924	validation_1-rmse:1065.98
[400]	validation_0-rmse:866.039	validation_1-rmse:1055.8
[500]	validation_0-rmse:842.007	validation_1-rmse:1030.84
[600]	validation_0-rmse:789.895	validation_1-rmse:975.336
[700]	validation_0-rmse:694.969	validation_1-rmse:871.571
[800]	validation_0-rmse:557.798	validation_1-rmse:716.699
[900]	validation_0-rmse:405.58	validation_1-rmse:542.434
[1000]	validation_0-rmse:276.35	validation_1-rmse:397.253
[1100]	validation_0-rmse:190.667	validation_1-rmse:305.207
[1200]	validation_0-rmse:154.971	validation_1-rmse:271.795
[1300]	validati

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 11.90it/s]


53 53
all_idx  : 13 27
train_idx: 13 23
valid_idx: 24 24
test_idx : 28 28
[0]	validation_0-rmse:900.66	validation_1-rmse:1451.31
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[100]	validation_0-rmse:900.128	validation_1-rmse:1450.78
[200]	validation_0-rmse:898.694	validation_1-rmse:1449.33
[300]	validation_0-rmse:894.87	validation_1-rmse:1445.44
[400]	validation_0-rmse:884.963	validation_1-rmse:1435.29
[500]	validation_0-rmse:860.829	validation_1-rmse:1410.04
[600]	validation_0-rmse:808.295	validation_1-rmse:1353.21
[700]	validation_0-rmse:712.128	validation_1-rmse:1245.94
[800]	validation_0-rmse:572.214	validation_1-rmse:1087.8
[900]	validation_0-rmse:415.731	validation_1-rmse:924.84
[1000]	validation_0-rmse:282.041	validation_1-rmse:806.888
[1100]	validation_0-rmse:193.642	validation_1-rmse:738.259
[1200]	validation_0-rmse:155.982	validation_1-rmse:712.916
[1300]	validati

In [113]:
for month in [21,22,23,24]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear','regMonth'] + stat_feat
    cate_feat = ['adcode','bodyType','model']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
#     print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.mt==month), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.mt==month), 'label'      ] = sub['forecastVolum'].values
    data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values
    
    
    
    
    
# sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
# sub.columns = ['id','forecastVolum']
# sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.07it/s]


all_idx  : 9 20
train_idx: 9 16
valid_idx: 17 17
test_idx : 21 21
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 128712	valid_1's l2: 98700.9
[200]	training's l2: 41958.5	valid_1's l2: 38841.5
[300]	training's l2: 23711.2	valid_1's l2: 28937.2
[400]	training's l2: 17316.6	valid_1's l2: 26511.4
[500]	training's l2: 13789.8	valid_1's l2: 25838.3
[600]	training's l2: 11344.6	valid_1's l2: 25413.5
[700]	training's l2: 9716.53	valid_1's l2: 25005.3
[800]	training's l2: 8521.85	valid_1's l2: 24786.1
[900]	training's l2: 7601.14	valid_1's l2: 24786.9
[1000]	training's l2: 6807.82	valid_1's l2: 24710.5
[1100]	training's l2: 6154.66	valid_1's l2: 24588.3
[1200]	training's l2: 5603.01	valid_1's l2: 24436.2
[1300]	training's l2: 5128.15	valid_1's l2: 24299.6
[1400]	training's l2: 4756.47	valid_1's l2: 24098.7
[1500]	training's l2: 4411.44	valid_1's l2: 23997.3
[1600]	training's l2: 4081.94	valid_1's l2: 23915.1
[1700]	training's l2: 3801.56	valid_1's l2: 23785

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.20it/s]


all_idx  : 9 21
train_idx: 9 17
valid_idx: 18 18
test_idx : 22 22
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 124896	valid_1's l2: 101622
[200]	training's l2: 41221.5	valid_1's l2: 40668.4
[300]	training's l2: 23551.2	valid_1's l2: 30796.4
[400]	training's l2: 17247.5	valid_1's l2: 28629.1
[500]	training's l2: 13644.1	valid_1's l2: 28447.5
[600]	training's l2: 11311.3	valid_1's l2: 28214.8
[700]	training's l2: 9665.8	valid_1's l2: 28306.6
Early stopping, best iteration is:
[602]	training's l2: 11275.5	valid_1's l2: 28202.4
0.7483585609319627
valid mean: 547.3275192104005
true  mean: 523.3159090909091
test  mean: 648.8015280661291
(1320, 2)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.75it/s]


all_idx  : 9 22
train_idx: 9 18
valid_idx: 19 19
test_idx : 23 23
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 121129	valid_1's l2: 101400
[200]	training's l2: 40304.7	valid_1's l2: 38675.6
[300]	training's l2: 23218.8	valid_1's l2: 27340.3
[400]	training's l2: 17179.7	valid_1's l2: 24853.9
[500]	training's l2: 13713.1	valid_1's l2: 23959.2
[600]	training's l2: 11471.2	valid_1's l2: 23400.8
[700]	training's l2: 9962.11	valid_1's l2: 22931.2
[800]	training's l2: 8881.21	valid_1's l2: 22567.1
[900]	training's l2: 8007.93	valid_1's l2: 22383.6
[1000]	training's l2: 7299.37	valid_1's l2: 22289.3
[1100]	training's l2: 6668.06	valid_1's l2: 22270.3
[1200]	training's l2: 6149.12	valid_1's l2: 22150.4
Early stopping, best iteration is:
[1196]	training's l2: 6170.05	valid_1's l2: 22128.1
0.7468931536269028
valid mean: 560.7094271141314
true  mean: 552.2295454545455
test  mean: 734.8046071367794
(1320, 2)


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  2.18it/s]


all_idx  : 9 23
train_idx: 9 19
valid_idx: 20 20
test_idx : 24 24
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 118772	valid_1's l2: 145395
[200]	training's l2: 39386.3	valid_1's l2: 51071.7
[300]	training's l2: 22711.1	valid_1's l2: 32006.2
[400]	training's l2: 17040	valid_1's l2: 27020.4
[500]	training's l2: 13774.6	valid_1's l2: 25125.4
[600]	training's l2: 11675.5	valid_1's l2: 24421
[700]	training's l2: 10208	valid_1's l2: 24129.7
[800]	training's l2: 9205.22	valid_1's l2: 23964.6
[900]	training's l2: 8388.5	valid_1's l2: 23799.9
[1000]	training's l2: 7727.56	valid_1's l2: 23768.7
Early stopping, best iteration is:
[989]	training's l2: 7801.96	valid_1's l2: 23734.8
0.7917019686244774
valid mean: 595.4352742803294
true  mean: 620.0212121212121
test  mean: 856.408786058386
(1320, 2)


In [68]:
for month in [21,22,23,24]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear','regMonth','popularity'] + stat_feat
    cate_feat = ['adcode','bodyType','model']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
#     print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
#     data.loc[(data.mt==(month)), 'pred_label'      ] = sub['forecastVolum'].values
    
    
    
    
    
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
# sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.37it/s]


all_idx  : 13 24
train_idx: 13 20
valid_idx: 21 21
test_idx : 25 25
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 86584.9	valid_1's l2: 151719
[200]	training's l2: 27784.3	valid_1's l2: 71704.5
[300]	training's l2: 14902.5	valid_1's l2: 57361.3
[400]	training's l2: 10243.5	valid_1's l2: 52224.7
[500]	training's l2: 7869.38	valid_1's l2: 50304.7
[600]	training's l2: 6385.06	valid_1's l2: 49569.7
[700]	training's l2: 5380.65	valid_1's l2: 48911.9
[800]	training's l2: 4634.41	valid_1's l2: 48274
[900]	training's l2: 4071.47	valid_1's l2: 48026.2
[1000]	training's l2: 3625	valid_1's l2: 47641.9
[1100]	training's l2: 3253.2	valid_1's l2: 47425.4
[1200]	training's l2: 2928.61	valid_1's l2: 47143.3
[1300]	training's l2: 2653.04	valid_1's l2: 46870.8
[1400]	training's l2: 2414.07	valid_1's l2: 46669.9
[1500]	training's l2: 2207.68	valid_1's l2: 46487.6
[1600]	training's l2: 2028.06	valid_1's l2: 46336.7
[1700]	training's l2: 1876.96	valid_1's l2: 46208.4
[

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.41it/s]


all_idx  : 13 25
train_idx: 13 21
valid_idx: 22 22
test_idx : 26 26
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 90083.5	valid_1's l2: 118412
[200]	training's l2: 28475.8	valid_1's l2: 59906.1
[300]	training's l2: 15309.6	valid_1's l2: 52080.3
[400]	training's l2: 10785.7	valid_1's l2: 50404.3
[500]	training's l2: 8324.54	valid_1's l2: 49632.1
[600]	training's l2: 6828.3	valid_1's l2: 49046.4
[700]	training's l2: 5813.9	valid_1's l2: 48997.8
Early stopping, best iteration is:
[648]	training's l2: 6301.83	valid_1's l2: 48762.4
0.7352932759641437


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.40it/s]


all_idx  : 13 26
train_idx: 13 22
valid_idx: 23 23
test_idx : 27 27
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 93591.6	valid_1's l2: 135289
[200]	training's l2: 29582.9	valid_1's l2: 52574.8
[300]	training's l2: 15864.8	valid_1's l2: 39181.2
[400]	training's l2: 11154.7	valid_1's l2: 36023.5
[500]	training's l2: 8739.67	valid_1's l2: 35331.5
[600]	training's l2: 7173.39	valid_1's l2: 34526.5
[700]	training's l2: 6126.06	valid_1's l2: 34402.5
[800]	training's l2: 5347.97	valid_1's l2: 34349
[900]	training's l2: 4770.82	valid_1's l2: 34426
Early stopping, best iteration is:
[806]	training's l2: 5308.31	valid_1's l2: 34325.1
0.7811424903161602


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.38it/s]


all_idx  : 13 27
train_idx: 13 23
valid_idx: 24 24
test_idx : 28 28
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 96023.9	valid_1's l2: 552065
[200]	training's l2: 29709.3	valid_1's l2: 354551
[300]	training's l2: 16067.9	valid_1's l2: 299015
[400]	training's l2: 11490.5	valid_1's l2: 282547
[500]	training's l2: 9067.98	valid_1's l2: 274322
[600]	training's l2: 7522.51	valid_1's l2: 271372
[700]	training's l2: 6477.28	valid_1's l2: 271395
Early stopping, best iteration is:
[658]	training's l2: 6879.88	valid_1's l2: 270441
0.6226761089064834


In [69]:
sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales0914.csv', index=False)

In [38]:
for month in [25,26,27,28]: 
    m_type = 'lgb' 
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear'] + stat_feat
    cate_feat = ['adcode','bodyType','model','regMonth']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)   
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
sub.columns = ['id','forecastVolum']
# sub[['id','forecastVolum']].round().astype(int).to_csv('CCF_sales.csv', index=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.38s/it]


53 53
all_idx  : 13 24
train_idx: 13 20
valid_idx: 21 21
test_idx : 25 25
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 7291.26	valid_1's l2: 37932.3
[200]	training's l2: 3715.93	valid_1's l2: 35716.9
[300]	training's l2: 2515.27	valid_1's l2: 35194.8
[400]	training's l2: 1831.38	valid_1's l2: 34788.8
[500]	training's l2: 1397.64	valid_1's l2: 34608.6
[600]	training's l2: 1065.51	valid_1's l2: 34527.1
[700]	training's l2: 849.095	valid_1's l2: 34415.2
[800]	training's l2: 679.238	valid_1's l2: 34333.1
[900]	training's l2: 545.247	valid_1's l2: 34326.4
Early stopping, best iteration is:
[838]	training's l2: 624.086	valid_1's l2: 34316.6
0.7559303097480222
valid mean: 602.8972278482273
true  mean: 649.3121212121212
test  mean: 493.9801201875369


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.35s/it]


53 53
all_idx  : 13 25
train_idx: 13 21
valid_idx: 22 22
test_idx : 26 26
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 7936.91	valid_1's l2: 41558.5
Early stopping, best iteration is:
[75]	training's l2: 10761.5	valid_1's l2: 41117.3
0.7414955122487169
valid mean: 623.4500376146684
true  mean: 616.5537878787878
test  mean: 324.6451813135614


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.36s/it]


53 53
all_idx  : 13 26
train_idx: 13 22
valid_idx: 23 23
test_idx : 27 27
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 8653.84	valid_1's l2: 32560.4
[200]	training's l2: 4627.91	valid_1's l2: 32133.2
Early stopping, best iteration is:
[155]	training's l2: 5757.15	valid_1's l2: 31853.4
0.7794578141087838
valid mean: 639.6227869473659
true  mean: 673.0143939393939
test  mean: 479.55100257280156


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.40s/it]


53 53
all_idx  : 13 27
train_idx: 13 23
valid_idx: 24 24
test_idx : 28 28
Training until validation scores don't improve for 100 rounds.
[100]	training's l2: 9306.24	valid_1's l2: 302435
[200]	training's l2: 5107.21	valid_1's l2: 292129
[300]	training's l2: 3583.18	valid_1's l2: 289373
[400]	training's l2: 2709.06	valid_1's l2: 288389
[500]	training's l2: 2162.77	valid_1's l2: 287983
[600]	training's l2: 1746.75	valid_1's l2: 287504
[700]	training's l2: 1446.22	valid_1's l2: 287029
[800]	training's l2: 1196.94	valid_1's l2: 286429
[900]	training's l2: 1008.26	valid_1's l2: 286030
Early stopping, best iteration is:
[856]	training's l2: 1089.91	valid_1's l2: 286004
0.588139278724323
valid mean: 645.3917897243205
true  mean: 899.8204545454546
test  mean: 468.0356962498267
